In [1]:
import pandas as pd
from pathlib import Path

In [2]:
train = Path('train')

In [3]:
test = Path('test')

In [4]:
# ใน train มี 7 พันไฟล์.  เอาที่เป็น ".xxx" ออก # โค้ดจาก https://colab.research.google.com/drive/1Ur-p2yQBf5JsUJbJJlZsm2VqNPY8gIkL
train_files = [f for f in train.iterdir() if not f.name.startswith('.')]
len(train_files)

3794

In [5]:
# ใน train มี 7 พันไฟล์.  เอาที่เป็น ".xxx" ออก
test_files = [f for f in test.iterdir() if not f.name.startswith('.')]
len(test_files)

483

In [6]:
list(set(train_files) & set(test_files)) # หาไฟล์ซ้ำ

[]

In [7]:
def readfile(path):
    _data = []
    with open(path,'r',encoding="utf-8-sig") as f:
        _temp = [i for i in f.read().split("\n\n") if i != '']
    for j in _temp:
        _temp_data = []
        for i in j.split("\n"):
            _t = i.strip().split("\t")
            _temp_data.append((_t[0],_t[1]))
        _data.append(_temp_data)
    return _data

In [60]:
#readfile(train_files[0])

In [61]:
#readfile(test_files[0])

In [8]:
train_data = []

In [9]:
for file in train_files:
    train_data.extend(readfile(file))

In [10]:
len(train_data)

63310

In [11]:
'' in train_data

False

In [12]:
#train_data[1000]

In [13]:
test_data = []

In [14]:
for file in test_files:
    test_data.extend(readfile(file))

In [15]:
len(test_data)

5250

In [16]:
import nltk

In [17]:
unigram_tagger = nltk.tag.UnigramTagger(train_data)

In [18]:
print("Accuracy : ", unigram_tagger.evaluate(test_data))

Accuracy :  0.9203002730632291


In [19]:
tagged_test_sentences = unigram_tagger.tag_sents([[token for token,tag in sent] for sent in test_data])
gold = [str(tag) for sentence in test_data for token,tag in sentence]
pred = [str(tag) for sentence in tagged_test_sentences for token,tag in sentence]
from sklearn import metrics
print(metrics.classification_report(gold, pred))

D:\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

          AJ       0.84      0.79      0.82      4403
          AV       0.74      0.70      0.72      6722
          AX       0.91      0.94      0.92      7556
          CC       0.86      0.97      0.91     17613
          CL       0.68      0.66      0.67      3739
          FX       0.98      0.98      0.98      6918
          IJ       1.00      0.50      0.67         4
          NG       1.00      1.00      1.00      1694
          NN       0.97      0.90      0.93     58568
          NU       0.98      0.92      0.95      6256
        None       0.00      0.00      0.00         0
          PA       0.80      0.90      0.85       194
          PR       0.81      0.88      0.84      2139
          PS       0.89      0.87      0.88     10886
          PU       1.00      1.00      1.00     37973
          VV       0.95      0.93      0.94     42586
          XX       0.00      0.00      0.00        27

    accuracy              

In [29]:
pt_tagger = nltk.tag.PerceptronTagger(load=False)
pt_tagger.train(train_data)

In [30]:
print("Accuracy : ", pt_tagger.evaluate(test_data))

Accuracy :  0.9623404316907728


In [58]:
tagged_test_sentences = pt_tagger.tag_sents([[token for token,tag in sent] for sent in test_data])
gold = [str(tag) for sentence in test_data for token,tag in sentence]
pred = [str(tag) for sentence in tagged_test_sentences for token,tag in sentence]
from sklearn import metrics
print(metrics.classification_report(gold, pred))

              precision    recall  f1-score   support

          AJ       0.90      0.86      0.88      4403
          AV       0.88      0.79      0.83      6722
          AX       0.95      0.94      0.95      7556
          CC       0.94      0.97      0.95     17613
          CL       0.87      0.85      0.86      3739
          FX       0.99      0.99      0.99      6918
          IJ       0.00      0.00      0.00         4
          NG       1.00      1.00      1.00      1694
          NN       0.97      0.98      0.97     58568
          NU       0.98      0.98      0.98      6256
          PA       0.84      0.87      0.86       194
          PR       0.87      0.85      0.86      2139
          PS       0.94      0.93      0.94     10886
          PU       1.00      1.00      1.00     37973
          VV       0.95      0.97      0.96     42586
          XX       0.00      0.00      0.00        27

    accuracy                           0.96    207278
   macro avg       0.82   

In [20]:
import pickle

In [35]:
# Store data (serialize)
with open('lst20_pt_tagger.pkl', 'wb') as handle:
    pickle.dump(pt_tagger, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [21]:
import json
unigram_tagger_json= json.dumps(unigram_tagger._context_to_tag, ensure_ascii=False)

In [22]:
# Store data (serialize)
with open('lst20_unigram_tagger.json', 'w',encoding = "utf-8-sig") as f:
    f.write(unigram_tagger_json)